In [1]:
import json
import os

import openai
from papermage import Document

from papermage_components.GPT_predictor import GPT_predictor

In [3]:
json_filename = "data/Midyear_Review_Papers_Parsed/microstructure and properties of 3d needle-punched carbon-silicon carbide brake materials.json"

with open(json_filename) as f:
  parsed_paper = Document.from_json(json.load(f))

In [4]:
parsed_paper

Document with 42 layers: ['symbols', 'images', 'metadata', 'tokens', 'rows', 'pages', 'reading_order_sections', 'words', 'sentences', 'Application', 'Result', 'Property', 'Phenomenon', 'Environment', 'Material', 'MStructure', 'Operation', 'Descriptor', 'Number', 'Amount_Unit', 'Synthesis', 'Participating_Material', 'Characterization', 'Microstructure', 'blocks', 'vila_entities', 'titles', 'paragraphs', 'authors', 'abstracts', 'keywords', 'sections', 'lists', 'bibliographies', 'equations', 'algorithms', 'figures', 'tables', 'captions', 'headers', 'footers', 'footnotes']

In [7]:
parsed_paper.tables[0]  

[]